In [1]:
import pandas as pd

import sys
sys.path.append('../python/')
import ana
import ap
import aux
import cfg
import db
import fio
import lex

corpus_id = cfg.CORPUS_ID_HMT
import numpy as np
# corpus and gender pair significance of syn and lcon involves rng for
# the permutation baselines; setting the seed keeps results stable
# as long as cells are run in the same order top to bottom
np.random.seed(0)

## Load basic data

In [2]:
db.connect(corpus_id)
# get wide table with basic data
df_bt = ap.load_data(cfg.NRM_SPK, ['gender'])
# get partner and non-partner speaker pairs
df_spk_pairs = db.pd_read_sql_query(sql_fname=cfg.SQL_SP_FNAME)
# get entropy weights
df_spk_pairs = lex.get_entropy_weights(corpus_id, df_spk_pairs)
# load lexica
dir_lem_types, dir_all_types, _ = lex.load_lex_w_lem(cfg.LEX_DIR_FNAME)
hdg_lem_types, _, _ = lex.load_lex_w_lem(cfg.LEX_HDG_FNAME)
geo_lem_types, geo_all_types, _ = lex.load_lex_w_lem(cfg.LEX_GEO_FNAME)
neg = lex.load_lex(cfg.LEX_NON_FNAME) + lex.load_lex(cfg.LEX_MAP_FNAME)
# get 25 most frequent types in the corpus 
# (ignoring non-words, landmarks, and directional words)
mf_types = lex.get_mf_types(corpus_id, neg=neg+dir_all_types)
db.close()

# Acoustic measures

## Compute measures

In [3]:
# local similarity
df_lsim = ap.lsim(df_bt)

In [4]:
# synchrony
df_syn = ap.syn(df_bt)

In [5]:
# local convergence
df_lcon = ap.lcon(df_bt)

In [6]:
# global similarity
df_gsim, df_gsim_raw = ap.gsim(df_bt, df_spk_pairs)

In [7]:
# global convergence
df_gcon, df_gcon_raw = ap.gcon(df_bt)

## Results

### Whole corpus

In [8]:
df_lsim.xs((0,0), level=(3,4)).iloc[[0]]

,,,intensity_mean,intensity_max,pitch_mean,pitch_max,jitter,shimmer,nhr,rate_syl
ses_type,ses_id,gender_pair,,,,,,,,
GAME,-1,0,"(2.950300700613202, 0.009927354878583938, 15, ...","(-1.6717711458438254, 0.11529417767273314, 15,...","(1.9724512806765884, 0.06728385577223092, 15, ...","(-0.8670503236842666, 0.39957249631594405, 15,...","(1.461174890973498, 0.16459825763708874, 15, -...","(0.3288691097111404, 0.7468013304820753, 15, -...","(3.5172037257848867, 0.003111723185325161, 15,...","(-0.9256655282308608, 0.3692737085423131, 15, ..."


In [9]:
ana.check_significance(
    df_syn.xs((0,0), level=(3,4)), 'all', '_', stat_idxs=[0,3])

,intensity_mean,intensity_max,pitch_mean,pitch_max,jitter,shimmer,nhr,rate_syl
_,,,,,,,,
all,"(0.5353760067137115, 0.600238139156069, 15)","(-3.78071423110861, 0.0018133500505258234, 15)","(1.8396533449868906, 0.08569355165612846, 15)","(-3.9199627127596908, 0.0013645359813184797, 15)","(1.1525338505646328, 0.26713913906292674, 15)","(0.5027206026905452, 0.622461010753253, 15)","(4.375223526858376, 0.0005432179728745163, 15)","(-1.4038146542594354, 0.18073821144523783, 15)"


In [10]:
ana.check_significance(
    df_lcon.xs((0,0), level=(3,4)), 'all', '_', stat_idxs=[0,3])

,intensity_mean,intensity_max,pitch_mean,pitch_max,jitter,shimmer,nhr,rate_syl
_,,,,,,,,
all,"(-1.3106775003998794, 0.20968038354815244, 15)","(-2.1937431735453794, 0.044422646659719095, 15)","(-0.8304498213744157, 0.41930655264919603, 15)","(1.3690387877066568, 0.19113948482287385, 15)","(1.5605682402559817, 0.13947227766778614, 15)","(0.4784160241921774, 0.6392524418932419, 15)","(0.7782902826190619, 0.4485014498668385, 15)","(0.7478924562322798, 0.46608958376580734, 15)"


In [11]:
df_gsim.iloc[[0]]

,,,intensity_mean,intensity_max,pitch_mean,pitch_max,jitter,shimmer,nhr,rate_syl
ses_type,tsk_or_ses,gender_pair,,,,,,,,
GAME,tsk,0,"(0.751624731792699, 0.4550755628930546, 63)","(0.33273103837820106, 0.74044199784432, 63)","(-1.1634189922186378, 0.24904711332845486, 63)","(-0.0077585062831473164, 0.9938341858078451, 63)","(0.027773781074242877, 0.9779303709640524, 63)","(-0.32207679782701165, 0.7484608328684952, 63)","(2.461307567700664, 0.016593968587096296, 63)","(0.16392705177172773, 0.870313348891263, 63)"


In [12]:
df_gcon.iloc[[0]]

,,,intensity_mean,intensity_max,pitch_mean,pitch_max,jitter,shimmer,nhr,rate_syl
ses_type,tsk_or_ses,gender_pair,,,,,,,,
GAME,tsk,0,"(-2.6536240539664733, 0.012448686071333504, 31)","(-2.6819007296229365, 0.011628660680809751, 31)","(-1.749478147619724, 0.09010365730581356, 31)","(-2.6136414219820305, 0.013700084313640351, 31)","(0.4658640747498466, 0.6445695502859167, 31)","(-0.4019351276929079, 0.6904895980860903, 31)","(-0.4738356336719122, 0.6389374228173912, 31)","(-1.6019051485277536, 0.11931904640694506, 31)"


### Per session (symmetric local measures only)

In [13]:
df = df_lsim.xs((0,0), level=(3,4)).iloc[4:].copy()
df['p_sig'] = df.apply(ana.bh, axis=1)
df.transpose()

ses_type                                                     GAME  \
ses_id                                                         0    
gender_pair                                                     M   
intensity_mean  (1.3794598223733943, 0.16920637064181823, 212,...   
intensity_max   (0.5650244079245067, 0.5726545775153646, 212, ...   
pitch_mean      (0.54585540912374, 0.5857392494349103, 212, -1...   
pitch_max       (0.6391290316414799, 0.5234289464666486, 212, ...   
jitter          (0.2045651675875968, 0.8381081300983003, 212, ...   
shimmer         (0.24628298762051454, 0.8057014719529567, 212,...   
nhr             (0.3992429866735837, 0.6901160623079002, 212, ...   
rate_syl        (-1.0154276189983114, 0.31105949316940956, 212...   
p_sig                                                           0   

ses_type                                                           \
ses_id                                                         1    
gender_pair                                                     M   
intensity_mean  (-0.2501898779634408, 0.8026100611941638, 303,...   
intensity_max   (-1.211413345541017, 0.2266809501580378, 303, ...   
pitch_mean      (0.16620016781460795, 0.8681102736131427, 303,...   
pitch_max       (0.4020210741911711, 0.6879519494742978, 303, ...   
jitter          (0.06829269891618125, 0.9455976802220694, 303,...   
shimmer         (-0.1285888802132625, 0.8977682737428252, 303,...   
nhr             (-0.1279340108889675, 0.8982860622323587, 303,...   
rate_syl        (-2.2221875702452447, 0.027008856533536966, 30...   
p_sig                                                           0   

ses_type                                                           \
ses_id                                                         2    
gender_pair                                                     X   
intensity_mean  (1.4722477565645191, 0.14170470011612343, 419,...   
intensity_max   (1.1580276591660326, 0.24751239852123058, 419,...   
pitch_mean      (-0.5249113408213725, 0.5999224434506263, 419,...   
pitch_max       (-1.533122234545014, 0.12600042217257507, 419,...   
jitter          (-0.165177338156293, 0.8688840635244534, 419, ...   
shimmer         (0.0074795992257140915, 0.9940357587088333, 41...   
nhr             (0.044550046058141744, 0.9644871839249727, 419...   
rate_syl        (0.35395660381313, 0.723549450673334, 419, -1....   
p_sig                                                           0   

ses_type                                                           \
ses_id                                                         3    
gender_pair                                                     X   
intensity_mean  (1.935464568382218, 0.053701165315506144, 367,...   
intensity_max   (-1.1031642587547528, 0.2706786753882968, 367,...   
pitch_mean      (-0.15104591452166188, 0.8800224847625719, 367...   
pitch_max       (-1.8346012408725945, 0.06737411242741707, 367...   
jitter          (0.5733242093042755, 0.5667763267094641, 367, ...   
shimmer         (2.16912113024579, 0.03071497199013687, 367, -...   
nhr             (1.3176695583596567, 0.1884362407444316, 367, ...   
rate_syl        (-1.649533655175903, 0.09989396014736365, 367,...   
p_sig                                                           0   

ses_type                                                           \
ses_id                                                         4    
gender_pair                                                     X   
intensity_mean  (-0.10564042892123664, 0.9160673007977644, 106...   
intensity_max   (-0.8967920747004576, 0.37186210954817556, 106...   
pitch_mean      (-0.4537818490370277, 0.6509135549155531, 106,...   
pitch_max       (0.26486521833161397, 0.7916276516365321, 106,...   
jitter          (-0.5904792052342775, 0.556126246656937, 106, ...   
shimmer         (-0.797197578013879, 0.4271183894180335, 106, ...   
nhr             (-1.0175635214573284, 0.31120235792836726, 106.

In [14]:
df = df_syn.xs((0,0), level=(3,4)).copy()
df['p_sig'] = df.apply(ana.bh, axis=1)
df.transpose()

ses_type                                                     GAME  \
ses_id                                                         0    
gender_pair                                                     M   
intensity_mean  (0.14361171760967475, 0.036217260276442516, 21...   
intensity_max   (0.18575769467884326, 0.006552024874354469, 21...   
pitch_mean      (0.028115385180211627, 0.6832761411256639, 211...   
pitch_max       (0.014748486990000774, 0.8305532102546827, 211...   
jitter          (0.030581144581562594, 0.6571910901752066, 211...   
shimmer         (0.006382932180778652, 0.9262145114452773, 211...   
nhr             (0.03403288420112296, 0.6213669595870946, 211,...   
rate_syl        (-0.09910091692164129, 0.14948020821013672, 21...   
p_sig                                                           0   

ses_type                                                           \
ses_id                                                         1    
gender_pair                                                     M   
intensity_mean  (-0.006607074513298894, 0.9086631319235543, 30...   
intensity_max   (-0.19928186787323846, 0.0004733377151153252, ...   
pitch_mean      (0.04240171192760242, 0.4613748803502183, 302,...   
pitch_max       (-0.007411922201194694, 0.8975945128307584, 30...   
jitter          (-0.0056160995214805075, 0.9223155054952377, 3...   
shimmer         (-0.07850423481214955, 0.17217854110200273, 30...   
nhr             (0.026017933182959058, 0.6513800431960767, 302...   
rate_syl        (-0.1253167243711835, 0.028918973864846778, 30...   
p_sig                                                 0.000473338   

ses_type                                                           \
ses_id                                                         2    
gender_pair                                                     X   
intensity_mean  (-0.007763418436624044, 0.8739596399716981, 41...   
intensity_max   (-0.07214559804064694, 0.1399271572588392, 418...   
pitch_mean      (-0.010118061378451795, 0.8362083394349941, 41...   
pitch_max       (-0.05906165347500624, 0.22710458446941764, 41...   
jitter          (-0.015407605821524156, 0.7528834511887762, 41...   
shimmer         (0.0015254235609798161, 0.9751349781269093, 41...   
nhr             (-0.014443934646547823, 0.7678825242158319, 41...   
rate_syl        (0.008175628193782685, 0.867327587803776, 418,...   
p_sig                                                           0   

ses_type                                                           \
ses_id                                                         3    
gender_pair                                                     X   
intensity_mean  (0.10444697066293646, 0.04525211234884026, 366...   
intensity_max   (-0.06558656542563475, 0.20939165053596406, 36...   
pitch_mean      (0.03451304398495896, 0.5092414979057207, 366,...   
pitch_max       (-0.1302563979208276, 0.012387341580663496, 36...   
jitter          (0.11020134559216521, 0.034577089846750675, 36...   
shimmer         (0.1438888730771934, 0.005687389212369037, 366...   
nhr             (0.09643692673875745, 0.06460283637316308, 366...   
rate_syl        (-0.07229878048620754, 0.16635096869316443, 36...   
p_sig                                                   0.0123873   

ses_type                                                           \
ses_id                                                         4    
gender_pair                                                     X   
intensity_mean  (-0.0847228934124505, 0.38558690489734676, 105...   
intensity_max   (-0.26122479564842377, 0.006572717600535754, 1...   
pitch_mean      (-0.09710175886500735, 0.31974381348093883, 10...   
pitch_max       (-0.10461116626655578, 0.2835665154816624, 105...   
jitter          (-0.1118848411552144, 0.25123334934788427, 105...   
shimmer         (-0.13183175712100262, 0.1758713697490532, 105...   
nhr             (-0.010134036955637738, 0.9174878253487854, 10.

In [15]:
df = df_lcon.xs((0,0), level=(3,4)).copy()
df['p_sig'] = df.apply(ana.bh, axis=1)
df.transpose()

ses_type                                                     GAME  \
ses_id                                                         0    
gender_pair                                                     M   
intensity_mean  (0.1372756333875608, 0.045374455688658354, 211...   
intensity_max   (0.09424958860354812, 0.1705352893274876, 211,...   
pitch_mean      (-0.13507504590129432, 0.048977542075663204, 2...   
pitch_max       (-0.14373864514934487, 0.03605111763387185, 21...   
jitter          (-0.09113019920362461, 0.18519313965010054, 21...   
shimmer         (-0.010599486138173162, 0.8777765528881205, 21...   
nhr             (-0.09908545150509179, 0.1495440465840884, 211...   
rate_syl        (0.03851310118497725, 0.5761752905042223, 211,...   
p_sig                                                           0   

ses_type                                                           \
ses_id                                                         1    
gender_pair                                                     M   
intensity_mean  (0.016102472566233414, 0.7797708118714729, 302...   
intensity_max   (-0.0213178386576732, 0.7112334272542608, 302,...   
pitch_mean      (0.04607377232483632, 0.4234562416183644, 302,...   
pitch_max       (0.10595828441324084, 0.06503286837770815, 302...   
jitter          (0.14501218278498304, 0.011362385452055198, 30...   
shimmer         (-0.09035203200802298, 0.1159346401165027, 302...   
nhr             (0.005067805119361661, 0.9298792561203334, 302...   
rate_syl        (0.08814236557168967, 0.1251602068686701, 302,...   
p_sig                                                           0   

ses_type                                                           \
ses_id                                                         2    
gender_pair                                                     X   
intensity_mean  (-0.13349161440723506, 0.006146099140312354, 4...   
intensity_max   (-0.22875160323613455, 2.169099034497554e-06, ...   
pitch_mean      (-0.0011562858566985124, 0.9811507786837599, 4...   
pitch_max       (0.15190208834115507, 0.0017967331907839704, 4...   
jitter          (-0.07437631780821304, 0.12805251770367, 418, ...   
shimmer         (-0.08433192391762846, 0.08430864591512097, 41...   
nhr             (-0.18722376695957096, 0.00011352967905313239,...   
rate_syl        (-0.07804653762024874, 0.11023268603358524, 41...   
p_sig                                                   0.0061461   

ses_type                                                           \
ses_id                                                         3    
gender_pair                                                     X   
intensity_mean  (-0.12546653709227026, 0.016031368321801475, 3...   
intensity_max   (-0.216228163885636, 2.8710809406688403e-05, 3...   
pitch_mean      (0.033062490906229876, 0.5272145969990828, 366...   
pitch_max       (0.02587321128029641, 0.6207917667522307, 366,...   
jitter          (-0.07849044075037634, 0.13286362954748854, 36...   
shimmer         (-0.04324069766832888, 0.40820005370112283, 36...   
nhr             (-0.034846292712280125, 0.5051579925191877, 36...   
rate_syl        (-0.06268968602728865, 0.23026143075165137, 36...   
p_sig                                                 2.87108e-05   

ses_type                                                           \
ses_id                                                         4    
gender_pair                                                     X   
intensity_mean  (-0.12109316931054986, 0.21406580476112877, 10...   
intensity_max   (-0.31391579129951924, 0.0009923831514666139, ...   
pitch_mean      (0.15666855242136227, 0.10706520376272737, 105...   
pitch_max       (-0.10589272067272144, 0.27767694297962936, 10...   
jitter          (0.04880380423391708, 0.6176394696724155, 105,...   
shimmer         (0.0926798811376721, 0.3423798965835887, 105, ...   
nhr             (0.023193445674346586, 0.8125568424275347, 105.

### Per gender pair

In [16]:
df_lsim.xs((0,0), level=(3,4)).iloc[1:4]

intensity_mean  \
ses_type ses_id gender_pair                                                      
GAME     -1     F            (1.960855995621649, 0.10717048828297132, 5, -1...   
                M            (2.5381207903655163, 0.08481884077218543, 3, -...   
                X            (0.7808684459545729, 0.4702165352415676, 5, -1...   

                                                                 intensity_max  \
ses_type ses_id gender_pair                                                      
GAME     -1     F            (-0.5586325155899052, 0.6005045470965885, 5, -...   
                M            (-0.79470195850723, 0.4848505939268746, 3, -1....   
                X            (-1.72742903569476, 0.14467005268089952, 5, -1...   

                                                                    pitch_mean  \
ses_type ses_id gender_pair                                                      
GAME     -1     F            (1.6212179132230287, 0.16589726951144407, 5, -...   
                M            (1.9603214003261313, 0.14480678899859298, 3, -...   
                X            (-0.5997778608484393, 0.5747971979577713, 5, -...   

                                                                     pitch_max  \
ses_type ses_id gender_pair                                                      
GAME     -1     F            (-0.8956457929630588, 0.41148241600217106, 5, ...   
                M            (1.7718810441379054, 0.1745360294841449, 3, -1...   
                X            (-0.8568496974682573, 0.4306704669107117, 5, -...   

                                                                        jitter  \
ses_type ses_id gender_pair                                                      
GAME     -1     F            (1.7776907952421972, 0.1355976209764034, 5, -1...   
                M            (2.733912409464263, 0.07170429417308696, 3, -1...   
                X            (-0.5580880527822764, 0.6008493137285371, 5, -...   

                                                                       shimmer  \
ses_type ses_id gender_pair                                                      
GAME     -1     F            (0.4160944478798556, 0.6945978028739609, 5, -1...   
                M            (-0.8009566474814103, 0.48172146809699395, 3, ...   
                X            (0.22457727108732506, 0.8311972130281714, 5, -...   

                                                                           nhr  \
ses_type ses_id gender_pair                                                      
GAME     -1     F            (7.4880107370289775, 0.0006712267597079906, 5,...   
                M            (2.19488915839774, 0.11572364668453462, 3, -1....   
                X            (0.6525691705142885, 0.542842064193864, 5, -1....   

                                                                      rate_syl  
ses_type ses_id gender_pair                                                     
GAME     -1     F            (-0.8004461384262674, 0.4597781148690467, 5, -...  
                M            (-0.904590226736237, 0.43236349727859114, 3, -...  
                X            (0.021323194982948268, 0.9838126170568616, 5, ...

In [17]:
ana.check_significance_gp(
    df_syn.xs((0,0), level=(3,4)), stat_idxs=[0,3])

,intensity_mean,intensity_max,pitch_mean,pitch_max,jitter,shimmer,nhr,rate_syl
gender_pair,,,,,,,,
F,"(1.6153986044504707, 0.16714599102673505, 5)","(-2.5517372778041927, 0.05115784430020692, 5)","(1.5524897287581707, 0.18125044122364817, 5)","(-4.114127576358027, 0.00922661427492291, 5)","(1.795288705169389, 0.13255921304004945, 5)","(1.2544588570881459, 0.2651228003993824, 5)","(7.658977231958101, 0.0006042636077522875, 5)","(-1.2321584821444471, 0.2726685414381813, 5)"
M,"(2.3215289786975055, 0.102940346125242, 3)","(-0.6341558803312112, 0.5710308637712771, 3)","(1.82303734258849, 0.1658171763052775, 3)","(-0.30947828509213104, 0.7772079123512604, 3)","(1.2270988669197387, 0.3072999121556139, 3)","(-0.7061086765108524, 0.5310170419797431, 3)","(2.7687914904978723, 0.06963872540236639, 3)","(-0.48149746167525304, 0.6631027807664538, 3)"
X,"(-1.4892874059872043, 0.19658928917278892, 5)","(-5.663821652354015, 0.0023858417819298395, 5)","(-0.339950801314825, 0.7477092133580105, 5)","(-2.557698674191475, 0.05078842362630696, 5)","(-0.33196034917445716, 0.7533827104199848, 5)","(-0.07770443667400449, 0.9410768901425461, 5)","(0.9463905918649811, 0.38740611229962413, 5)","(-0.5620519546156928, 0.5983419878282998, 5)"


In [18]:
ana.check_significance_gp(
    df_lcon.xs((0,0), level=(3,4)), stat_idxs=[0,3])

,intensity_mean,intensity_max,pitch_mean,pitch_max,jitter,shimmer,nhr,rate_syl
gender_pair,,,,,,,,
F,"(0.13360009115358393, 0.8989295667255559, 5)","(-0.24847997506358027, 0.8136461878223835, 5)","(-1.1234147365472344, 0.3122870820257406, 5)","(1.7836930904384192, 0.13455340986340467, 5)","(1.2088428438510053, 0.28076454338317247, 5)","(1.1685022478217724, 0.2952811518252715, 5)","(2.196277143260701, 0.07946564688355408, 5)","(1.9605658077877997, 0.10721029440052904, 5)"
M,"(0.17471676067900382, 0.8724281454204372, 3)","(-0.5420552307043598, 0.6254519637889722, 3)","(-0.9564363332502542, 0.40941076817766225, 3)","(0.29990262565104403, 0.7838307646980067, 3)","(0.5805708943253696, 0.6022657710464905, 3)","(0.43146567302937855, 0.6952566470792831, 3)","(0.20544834285486624, 0.8503727477083998, 3)","(0.9986061931619029, 0.3915789551868118, 3)"
X,"(-4.756082428235638, 0.005077040573478501, 5)","(-4.953310086925282, 0.0042727249720928865, 5)","(0.4737136546439702, 0.6556664736725284, 5)","(0.2402835271429145, 0.8196512590602592, 5)","(0.7067622463653691, 0.511280632048585, 5)","(-2.0615604256079245, 0.09424590976854075, 5)","(-0.6495580242532609, 0.544632977309784, 5)","(-0.7147205176288193, 0.5067539080257301, 5)"


In [19]:
df_gsim.iloc[1:4]

intensity_mean  \
ses_type tsk_or_ses gender_pair                                                  
GAME     tsk        F            (-0.1512272199732009, 0.8811150591339684, 23)   
                    M             (1.613552354957009, 0.12745988315475684, 15)   
                    X            (0.06834278973432444, 0.9461034441229759, 23)   

                                                                   intensity_max  \
ses_type tsk_or_ses gender_pair                                                    
GAME     tsk        F            (5.475218388108705e-16, 0.9999999999999996, 23)   
                    M               (0.6793496846271456, 0.5072647646121624, 15)   
                    X            (4.571107813078957e-16, 0.9999999999999997, 23)   

                                                                     pitch_mean  \
ses_type tsk_or_ses gender_pair                                                   
GAME     tsk        F            (-1.6032066963670881, 0.12253473217926532, 23)   
                    M             (0.30273702776164224, 0.7662480291011211, 15)   
                    X             (-0.6147131163917686, 0.5447782149475809, 23)   

                                                                      pitch_max  \
ses_type tsk_or_ses gender_pair                                                   
GAME     tsk        F            (-1.1426193107194162, 0.26495203445308263, 23)   
                    M             (1.5065266089878204, 0.15270002880795014, 15)   
                    X             (0.10144885226848134, 0.9200733865074922, 23)   

                                                                        jitter  \
ses_type tsk_or_ses gender_pair                                                  
GAME     tsk        F            (1.2044610435528573, 0.24066002394701025, 23)   
                    M            (-0.6106171364177642, 0.5505938594605939, 15)   
                    X            (-0.4333752961175933, 0.6687795680308588, 23)   

                                                                        shimmer  \
ses_type tsk_or_ses gender_pair                                                   
GAME     tsk        F              (1.242355896902018, 0.22662370663906553, 23)   
                    M            (-1.6022531497344306, 0.12994461886855818, 15)   
                    X            (-0.45752761548056264, 0.6515845402972226, 23)   

                                                                            nhr  \
ses_type tsk_or_ses gender_pair                                                   
GAME     tsk        F            (3.0581453105937713, 0.005572819348676005, 23)   
                    M            (-3.405427012202431, 0.003913954083462373, 15)   
                    X               (1.6022570997536398, 0.122745100480272, 23)   

                                                                      rate_syl  
ses_type tsk_or_ses gender_pair                                                 
GAME     tsk        F            (0.22144983058385828, 0.8266965145137879, 23)  
                    M             (-0.635278610971052, 0.5348185410342634, 15)  
                    X            (0.30053330185610977, 0.7664701897609802, 23)

In [20]:
df_gcon.iloc[1:]

intensity_mean  \
ses_type tsk_or_ses gender_pair                                                   
GAME     tsk        F             (-0.9528766356150495, 0.3611222618804184, 11)   
                    M             (-1.3158185005081928, 0.22968805459251396, 7)   
                    X            (-2.1252234105698196, 0.05705063688363686, 11)   

                                                                  intensity_max  \
ses_type tsk_or_ses gender_pair                                                   
GAME     tsk        F            (-1.3644335753739023, 0.19969356174579847, 11)   
                    M               (-1.1730387584707391, 0.279146324522841, 7)   
                    X            (-1.9419398373700318, 0.07818305853381151, 11)   

                                                                     pitch_mean  \
ses_type tsk_or_ses gender_pair                                                   
GAME     tsk        F            (-1.1413379883399397, 0.27796735506472225, 11)   
                    M               (-0.6033167217084315, 0.565326121176805, 7)   
                    X             (-1.1264244204891745, 0.2839601041958964, 11)   

                                                                     pitch_max  \
ses_type tsk_or_ses gender_pair                                                  
GAME     tsk        F            (-2.238499515198997, 0.04682418694361484, 11)   
                    M             (-1.110902832480613, 0.30330564279435057, 7)   
                    X             (-1.1176418131655046, 0.287535646832244, 11)   

                                                                       jitter  \
ses_type tsk_or_ses gender_pair                                                 
GAME     tsk        F            (1.515463559666049, 0.15785060152688352, 11)   
                    M             (-0.4123973848155351, 0.692385321585134, 7)   
                    X             (-0.3839800971375035, 0.70831176318055, 11)   

                                                                       shimmer  \
ses_type tsk_or_ses gender_pair                                                  
GAME     tsk        F             (1.1105654491349222, 0.2904416772657065, 11)   
                    M               (1.1163016662540386, 0.301140724924687, 7)   
                    X            (-2.321674334585933, 0.04045830573504206, 11)   

                                                                            nhr  \
ses_type tsk_or_ses gender_pair                                                   
GAME     tsk        F             (0.8487374340959367, 0.41411615652713263, 11)   
                    M             (-0.9724630404798131, 0.36321696298581524, 7)   
                    X            (-0.7431878300016684, 0.47294244391735785, 11)   

                                                                      rate_syl  
ses_type tsk_or_ses gender_pair                                                 
GAME     tsk        F            (-2.938733425945383, 0.01347824215968191, 11)  
                    M              (1.1474426120178443, 0.2888986523306568, 7)  
                    X             (-0.5527915034030028, 0.591462418262753, 11)

### Comparison of gender pairs

In [21]:
ana.compare_gps(df_lsim.xs((0,0), level=(3,4)).iloc[4:], stat_idx=3)

,intensity_mean,intensity_max,pitch_mean,pitch_max,jitter,shimmer,nhr,rate_syl
comp,,,,,,,,
F:M,"(-1.3110850181804947, 0.22620879014279305, 8)","(-0.30023503884442726, 0.7716516345147294, 8)","(-0.3844078090457309, 0.7106929171343364, 8)","(-1.082676928960505, 0.31050236726026975, 8)","(0.6057225958533756, 0.561486225589342, 8)","(0.8584014691524624, 0.4156473957993123, 8)","(0.8970826644789507, 0.3958680139582923, 8)","(0.0976557651393754, 0.9246080083727961, 8)"
F:X,"(0.6676735400337895, 0.5194527471640757, 10)","(-0.13311291680154558, 0.8967446506567058, 10)","(0.7131334803526974, 0.4920692815479868, 10)","(-0.1302808437865784, 0.8989277104173639, 10)","(0.979495923434934, 0.3504368139022883, 10)","(0.48055410996978576, 0.64117326292245, 10)","(1.0729836282523892, 0.30848842153918427, 10)","(-0.8949354067951498, 0.3918532694503576, 10)"
M:X,"(1.556730412347457, 0.15814748240429197, 8)","(0.26198219992575755, 0.7999545247135996, 8)","(1.0567350924742052, 0.32149602855453296, 8)","(3.5049617359909155, 0.008020555527176737, 8)","(0.4057562593662979, 0.6955573679985623, 8)","(-0.1451858965036642, 0.8881553804976815, 8)","(0.3218231557798968, 0.755834821673405, 8)","(-1.0395578967802712, 0.3289411888181296, 8)"


In [22]:
ana.compare_gps(df_syn.xs((0,0), level=(3,4)), stat_idx=0, func=aux.r2z)

,intensity_mean,intensity_max,pitch_mean,pitch_max,jitter,shimmer,nhr,rate_syl
comp,,,,,,,,
F:M,"(-0.7107697739868595, 0.49742230639991714, 8)","(-0.3031457079443257, 0.7695122607562952, 8)","(-1.6260050002349309, 0.14260138320690896, 8)","(-2.8863296044558093, 0.02031323567569025, 8)","(0.9171189703085161, 0.3858935438425871, 8)","(1.1003079337777424, 0.303201417956301, 8)","(1.826000493210247, 0.10528020497276476, 8)","(-0.4093148315900706, 0.6930482504220686, 8)"
F:X,"(2.2619833238261604, 0.04720920089546193, 10)","(1.8737969610383935, 0.09044172700729179, 10)","(1.0766296662660582, 0.3069331924348425, 10)","(-1.008716292543573, 0.3368948193305654, 10)","(1.3794380312377026, 0.19782415146554702, 10)","(0.8005892536552901, 0.44197449384467435, 10)","(3.1090776184922895, 0.011078064489826154, 10)","(-0.6800371639709998, 0.5119161923613184, 10)"
M:X,"(2.5844683816718432, 0.03239047977750462, 8)","(1.2949529792454357, 0.23145638165750101, 8)","(2.3903570688424223, 0.04383140099012981, 8)","(2.011320786081383, 0.07911605595860277, 8)","(0.6347252156594324, 0.5433290427160158, 8)","(-0.147413519116138, 0.8864531323515031, 8)","(1.2930385501757062, 0.2320859372817261, 8)","(-0.20409977069330723, 0.8433720403692195, 8)"


In [23]:
ana.compare_gps(df_lcon.xs((0,0), level=(3,4)), stat_idx=0, func=aux.r2z)

,intensity_mean,intensity_max,pitch_mean,pitch_max,jitter,shimmer,nhr,rate_syl
comp,,,,,,,,
F:M,"(-0.0860720237770075, 0.9335243920296596, 8)","(0.10995519669240958, 0.9151531620003337, 8)","(-0.38174016018673496, 0.7125940297498765, 8)","(0.5741054178522413, 0.5816745332967816, 8)","(0.040357353996572295, 0.968797229985458, 8)","(0.48737302535605953, 0.639068041943793, 8)","(0.931923325820386, 0.3786423962329981, 8)","(0.7794042254001164, 0.45817597066260063, 8)"
F:X,"(2.251815876131208, 0.04803136254003875, 10)","(2.1334955439520096, 0.058674692908895876, 10)","(-2.0837589523936653, 0.06379405627940385, 10)","(1.1548330815514558, 0.2750084603739563, 10)","(0.7064421286422515, 0.4960431086795677, 10)","(1.8654583225625434, 0.09168956280044775, 10)","(1.6835009779746783, 0.12318571825528774, 10)","(1.5607977159761361, 0.1496332008899898, 10)"
M:X,"(2.4577620306360175, 0.039456215266466685, 8)","(2.5581801544455427, 0.03374238251870353, 8)","(-1.2044227775915473, 0.2628470496476852, 8)","(0.39104346551696917, 0.7059733458248256, 8)","(0.5549807871298796, 0.5940809181606679, 8)","(0.5935123353830448, 0.5692345971507565, 8)","(0.6186059948256464, 0.553377275397317, 8)","(0.93696629664363, 0.3761953840795619, 8)"


In [24]:
df_tmp = df_gsim_raw.xs(0, level=4).reset_index(level=3)
df_tmp = df_tmp[df_tmp['tsk_id'] != 0].drop(['tsk_id'], axis=1)
ana.compare_gps(df_tmp, cols=[f + '_sim_nrm' for f in cfg.FEATURES])

,intensity_mean_sim_nrm,intensity_max_sim_nrm,pitch_mean_sim_nrm,pitch_max_sim_nrm,jitter_sim_nrm,shimmer_sim_nrm,nhr_sim_nrm,rate_syl_sim_nrm
comp,,,,,,,,
F:M,"(-0.8141199339923154, 0.42621595099130927, 18)","(-0.19387400177729427, 0.8484456397005921, 18)","(-0.6617933324046654, 0.5164871712246286, 18)","(-1.3342108281817633, 0.1987699053628036, 18)","(1.1450943272273615, 0.2671679495501639, 18)","(1.7105324352431563, 0.10434712779321066, 18)","(3.5501348306472176, 0.0022872545120252255, 18)","(1.1032453415785477, 0.28445246580938127, 18)"
F:X,"(0.1178294196944286, 0.907271993051036, 22)","(0.29819672183512697, 0.7683511554041861, 22)","(-0.297029070191495, 0.769230689149987, 22)","(-0.34116402061930334, 0.736216959677437, 22)","(1.40967729766315, 0.17261563994731555, 22)","(1.304474152504035, 0.20555441025005744, 22)","(0.6376400574217599, 0.5302891093921821, 22)","(-0.4840246715815774, 0.6331516914883806, 22)"
M:X,"(0.856984345935723, 0.4027194517292879, 18)","(0.5006511129196007, 0.6226831065009419, 18)","(0.37049797770502463, 0.7153354497260529, 18)","(0.621683027586373, 0.5419448910250391, 18)","(0.7740618979249548, 0.4489423444009909, 18)","(-0.5874572754524432, 0.5641940190370154, 18)","(-2.696166763519678, 0.014771404730462722, 18)","(-1.1664300174877804, 0.2586629754715736, 18)"


In [25]:
df_tmp = df_gcon_raw.xs(0, level=4).reset_index(level=3)
df_tmp = df_tmp[df_tmp['tsk_id'] != 0].drop(['tsk_id'], axis=1)
ana.compare_gps(df_tmp, cols=[f + '_con' for f in cfg.FEATURES])

,intensity_mean_con,intensity_max_con,pitch_mean_con,pitch_max_con,jitter_con,shimmer_con,nhr_con,rate_syl_con
comp,,,,,,,,
F:M,"(0.7467986308101965, 0.464828787152218, 18)","(0.40183409049125723, 0.692537230706336, 18)","(-0.3473193905822459, 0.7323793431913362, 18)","(-0.6169180698572834, 0.5450137968282498, 18)","(1.2340810560447641, 0.23304322482042045, 18)","(0.13979570803825594, 0.8903737609690092, 18)","(1.2940936143540522, 0.2119912389125274, 18)","(-2.747023702293318, 0.01325624199352568, 18)"
F:X,"(1.1354303758503541, 0.26841402724399843, 22)","(0.7533329169807875, 0.45923656539154467, 22)","(-0.13132552868796904, 0.8967113632557215, 22)","(-0.588604271932329, 0.5621218866907618, 22)","(1.4015660892799804, 0.17499559754493785, 22)","(2.4408880575535035, 0.02316032920966772, 22)","(1.1163796897723286, 0.27630487708054, 22)","(-0.9749813993610288, 0.34016894177957746, 22)"
M:X,"(0.21518028669936493, 0.8320460227303501, 18)","(0.22421944890774015, 0.8251119364614855, 18)","(0.25518046749223655, 0.8014766480827872, 18)","(0.02384078061687463, 0.9812419222294996, 18)","(-0.1820700943079676, 0.8575624653455416, 18)","(2.3050910652926726, 0.03328329949943103, 18)","(-0.19366008444634922, 0.8486106687277153, 18)","(0.7732637699589503, 0.4494026225502121, 18)"


### Individual gender comparisons

In [26]:
ana.compare_genders(df_lsim, df_bt, stat_idx=3)

,intensity_mean,intensity_max,pitch_mean,pitch_max,jitter,shimmer,nhr,rate_syl
comp,,,,,,,,
f:m,"(-1.0905720593035724, 0.28414345496934246, 30)","(-0.5828963316594375, 0.5643193255793585, 30)","(-0.5499714249759929, 0.5864106566683664, 30)","(-1.5372914107975546, 0.13470338574285098, 30)","(-0.44993539088831697, 0.6559914789431274, 30)","(0.000445963417140102, 0.9996471250498747, 30)","(-0.6188594672835316, 0.5406801160945749, 30)","(-0.9112896463033741, 0.36940752991241543, 30)"


In [27]:
ana.compare_genders(df_syn, df_bt, stat_idx=0, func=aux.r2z)

,intensity_mean,intensity_max,pitch_mean,pitch_max,jitter,shimmer,nhr,rate_syl
comp,,,,,,,,
f:m,"(-0.7175214976513745, 0.4786026505253467, 30)","(-0.5054736138640843, 0.616920053489922, 30)","(-0.6320837267258717, 0.5321200789494729, 30)","(-1.9695463478593254, 0.05818532617497313, 30)","(-0.11620995714463443, 0.9082605913805535, 30)","(0.30605079928093765, 0.7616798398763356, 30)","(0.014980338828034203, 0.988147049737321, 30)","(-1.3849832710519085, 0.17627134377195594, 30)"


In [28]:
ana.compare_genders(df_lcon, df_bt, stat_idx=0, func=aux.r2z)

,intensity_mean,intensity_max,pitch_mean,pitch_max,jitter,shimmer,nhr,rate_syl
comp,,,,,,,,
f:m,"(0.04410907758885907, 0.965109749618281, 30)","(0.401821752023198, 0.6906636057546248, 30)","(0.15581102049636428, 0.8772257054458577, 30)","(0.36416941768190564, 0.7182857542237844, 30)","(0.008379741459908148, 0.9933694893405671, 30)","(0.9753141000025536, 0.3372038689892487, 30)","(1.094098098466409, 0.28261919088558973, 30)","(0.4472108717218168, 0.6579351006857699, 30)"


In [29]:
ana.compare_partner_genders(df_lsim, df_bt, stat_idx=3)

,intensity_mean,intensity_max,pitch_mean,pitch_max,jitter,shimmer,nhr,rate_syl
comp,,,,,,,,
f:m,"(-0.24425379655993865, 0.8086970830871466, 30)","(-0.10653781844940774, 0.9158648988147327, 30)","(-0.18087875774922915, 0.8576791926763065, 30)","(-0.6256377130882029, 0.5362835484751445, 30)","(1.6417705549897412, 0.11108101051060856, 30)","(1.2219756282251373, 0.2312318814063315, 30)","(2.2179240561220794, 0.03427656481691549, 30)","(0.8378212985712534, 0.4087566307130116, 30)"


In [30]:
ana.compare_partner_genders(df_syn, df_bt, stat_idx=0, func=aux.r2z)

,intensity_mean,intensity_max,pitch_mean,pitch_max,jitter,shimmer,nhr,rate_syl
comp,,,,,,,,
f:m,"(-0.014727820860344071, 0.9883468354307057, 30)","(-0.009245248115947773, 0.992684672262282, 30)","(-1.1702023141835718, 0.25113212767266646, 30)","(-1.8568804923121613, 0.07317163358357705, 30)","(1.6457185348044037, 0.1102602887381019, 30)","(1.4889525882852555, 0.14693807904022957, 30)","(2.339807532561787, 0.02613992914829911, 30)","(0.4542567178681393, 0.6529137408713532, 30)"


In [31]:
ana.compare_partner_genders(df_lcon, df_bt, stat_idx=0, func=aux.r2z)

,intensity_mean,intensity_max,pitch_mean,pitch_max,jitter,shimmer,nhr,rate_syl
comp,,,,,,,,
f:m,"(0.396588940211193, 0.6944777031470988, 30)","(0.4679121084130548, 0.643229017479247, 30)","(-1.3462257448929063, 0.18831455192849234, 30)","(1.0203700278821104, 0.315711455853544, 30)","(0.26739732816786965, 0.7909930314627227, 30)","(0.6410097861134778, 0.5263832760516749, 30)","(0.8318671387587833, 0.4120568447458046, 30)","(0.9399243426242723, 0.35476335386737545, 30)"


### Role comparisons

In [32]:
ana.compare_roles(df_lsim, df_bt, stat_idx=3)

,intensity_mean,intensity_max,pitch_mean,pitch_max,jitter,shimmer,nhr,rate_syl
comp,,,,,,,,
d:f,"(1.487838245477201, 0.14186224274481835, 62)","(1.2073434404059382, 0.23188577625357956, 62)","(0.5745884723817477, 0.5676487530374172, 62)","(-0.8090123197821626, 0.42160183709972243, 62)","(0.3892619422216854, 0.69841619399806, 62)","(-0.4899848925040161, 0.6258740043370851, 62)","(-0.7043389864328042, 0.483858975731117, 62)","(1.3155023428067185, 0.1931852235434639, 62)"


In [33]:
ana.compare_roles(df_syn, df_bt, stat_idx=0, func=aux.r2z)

,intensity_mean,intensity_max,pitch_mean,pitch_max,jitter,shimmer,nhr,rate_syl
comp,,,,,,,,
d:f,"(0.015147655903352573, 0.9879630209355825, 62)","(0.4477720144213187, 0.6558784362282055, 62)","(0.2814397474802737, 0.779310334466307, 62)","(-0.5990389217304887, 0.5513295763273152, 62)","(-0.6469357984759987, 0.5200615073186609, 62)","(-0.24698674108736837, 0.8057345001124754, 62)","(-0.38956407103058605, 0.6981938824196139, 62)","(0.9047935865340969, 0.36907728633234727, 62)"


In [34]:
ana.compare_roles(df_lcon, df_bt, stat_idx=0, func=aux.r2z)

,intensity_mean,intensity_max,pitch_mean,pitch_max,jitter,shimmer,nhr,rate_syl
comp,,,,,,,,
d:f,"(1.7540054053677938, 0.08437119186964889, 62)","(0.4476169010291316, 0.6559897848127443, 62)","(-0.09786607550854742, 0.9223543430208183, 62)","(0.12957334350599356, 0.8973234490312576, 62)","(0.8375578496779963, 0.4054961856828908, 62)","(0.8932769263532916, 0.37516318004526494, 62)","(0.4314164099238121, 0.6676620619268081, 62)","(-1.1493509638185206, 0.25482580875803884, 62)"


### Influence of familiarity

In [35]:
ana.compare_familiarities(df_lsim, df_bt, 'ses', stat_idx=3)

,intensity_mean,intensity_max,pitch_mean,pitch_max,jitter,shimmer,nhr,rate_syl
comp,,,,,,,,
hi:lo,"(0.55155227418128, 0.5899507086680922, 14)","(0.4284965988002857, 0.6748089711037876, 14)","(-1.7526090650028134, 0.10153020949580288, 14)","(0.8481103427160461, 0.41064284360147796, 14)","(-1.0691551970339832, 0.3030880861331102, 14)","(-0.3128774401329481, 0.758985999342187, 14)","(-0.6066386235108958, 0.5538015324837985, 14)","(-0.9437839659271091, 0.3612835461179522, 14)"


In [36]:
ana.compare_familiarities(
    df_syn, df_bt, 'ses', stat_idx=0, func=aux.r2z)

,intensity_mean,intensity_max,pitch_mean,pitch_max,jitter,shimmer,nhr,rate_syl
comp,,,,,,,,
hi:lo,"(0.5035337267084006, 0.6224188327177558, 14)","(0.5175279793802492, 0.6128676810253741, 14)","(-0.2086627459807465, 0.8377169246354765, 14)","(0.4816940091677296, 0.6374660764955198, 14)","(-0.32354959453802196, 0.7510615802557491, 14)","(-0.6870812186572143, 0.5032549636802865, 14)","(-1.1658943996229914, 0.2631360696194321, 14)","(-2.107593802382174, 0.05357476512557833, 14)"


In [37]:
ana.compare_familiarities(
    df_lcon, df_bt, 'ses', stat_idx=0, func=aux.r2z)

,intensity_mean,intensity_max,pitch_mean,pitch_max,jitter,shimmer,nhr,rate_syl
comp,,,,,,,,
hi:lo,"(-0.9728884240969998, 0.34711930860569007, 14)","(-1.1845022878098026, 0.2559341488941441, 14)","(1.3024443548959803, 0.2137838557231886, 14)","(-0.10941651615817884, 0.9144250184903915, 14)","(-1.5024370834491712, 0.15520174271939025, 14)","(-2.510142437436533, 0.024972317478440795, 14)","(-1.8139951396334844, 0.09116342603567301, 14)","(0.38205086951646167, 0.7081602953494062, 14)"


In [38]:
ana.compare_familiarities(
    df_gsim_raw, df_bt, 'tsk', [c + '_sim_nrm' for c in cfg.FEATURES])

,intensity_mean_sim_nrm,intensity_max_sim_nrm,pitch_mean_sim_nrm,pitch_max_sim_nrm,jitter_sim_nrm,shimmer_sim_nrm,nhr_sim_nrm,rate_syl_sim_nrm
comp,,,,,,,,
hi:lo,"(0.3487130923340456, 0.7297402498337118, 30)","(0.15318955928757405, 0.8792745588836561, 30)","(1.9674578670451222, 0.058436747684643145, 30)","(-0.028825921960295924, 0.9771943386562305, 30)","(-0.30828643637248576, 0.7599948988244569, 30)","(-1.085746407123695, 0.2862389874811557, 30)","(-0.5187889477743987, 0.6077139549884742, 30)","(-1.1382034227327382, 0.2640439626058566, 30)"


In [39]:
ana.compare_familiarities(
    df_gcon_raw, df_bt, 'tsk', [c + '_con' for c in cfg.FEATURES])

,intensity_mean_con,intensity_max_con,pitch_mean_con,pitch_max_con,jitter_con,shimmer_con,nhr_con,rate_syl_con
comp,,,,,,,,
hi:lo,"(-0.5380927574492413, 0.5944830339102791, 30)","(-0.5282361817719086, 0.6012215137789851, 30)","(-0.6114577464625114, 0.545502622509529, 30)","(0.9156051219068322, 0.3671755903201298, 30)","(-0.41988688666302604, 0.677560100943772, 30)","(0.43871113684016333, 0.6640142333009322, 30)","(-0.4765447575543039, 0.6371390942534432, 30)","(1.1586701155832284, 0.2557311231170869, 30)"


# Lexical Measures

## Compute measures

In [40]:
df_mf, df_mf_raw = lex.dist_sim(
    corpus_id, df_spk_pairs, types_id='mf', types=mf_types)

In [41]:
df_dir, df_dir_raw = lex.dist_sim(
    corpus_id, df_spk_pairs, types_id='dir', types=dir_lem_types)

In [42]:
df_geo, df_geo_raw = lex.dist_sim(
    corpus_id, df_spk_pairs, types_id='geo', types=geo_lem_types)

In [43]:
df_hdg, df_hdg_raw = lex.dist_sim(
    corpus_id, df_spk_pairs, types_id='hdg', types=hdg_lem_types)

In [44]:
df_ppl, df_ppl_raw = lex.ppl(corpus_id, df_spk_pairs)

## Results

### Whole corpus

In [45]:
df_mf[['wgh']]

,wgh
_,
0,"(4.002889458676664, 0.0003623824603086523, 31)"


In [46]:
df_dir[['wgh']]

,wgh
_,
0,"(5.0057441644169, 2.1127175859648817e-05, 31)"


In [47]:
df_geo[['wgh']]

,wgh
_,
0,"(4.580983077568202, 7.111400518301427e-05, 31)"


In [48]:
df_hdg[['wgh']]

,wgh
_,
0,"(1.8426067668461021, 0.07496872454036499, 31)"


In [49]:
df_ppl[['wgh']]

,wgh
_,
0,"(3.769366035485885, 0.000690445281610602, 31)"


### Per gender pair

In [50]:
ana.check_significance_gp(
    df_mf_raw.xs((0,0), level=(3,4)), ['dsim_wgh',])

,dsim_wgh
gender_pair,
F,"(2.1763836897723805, 0.08148389636135177, 5)"
M,"(11.747760094376794, 0.0013255356452945705, 3)"
X,"(1.0706965613281083, 0.33324480945206864, 5)"


In [51]:
ana.check_significance_gp(
    df_dir_raw.xs((0,0), level=(3,4)), ['dsim_wgh',])

,dsim_wgh
gender_pair,
F,"(5.760057096835275, 0.002214329699449085, 5)"
M,"(3.9412836258591226, 0.029111674943061983, 3)"
X,"(1.6578894828075854, 0.15823823077041083, 5)"


In [52]:
ana.check_significance_gp(
    df_geo_raw.xs((0,0), level=(3,4)), ['dsim_wgh',])

,dsim_wgh
gender_pair,
F,"(3.7654812158001087, 0.013083548834036611, 5)"
M,"(0.4741319634662935, 0.6677784502093805, 3)"
X,"(2.923075223518861, 0.0328960557716538, 5)"


In [53]:
ana.check_significance_gp(
    df_hdg_raw.xs((0,0), level=(3,4)), ['dsim_wgh',])

,dsim_wgh
gender_pair,
F,"(0.5811701407956381, 0.58633780893448, 5)"
M,"(0.771220992135498, 0.49675109780126625, 3)"
X,"(0.9937533529526491, 0.3659705699013335, 5)"


In [54]:
ana.check_significance_gp(
    df_ppl_raw.xs((0,0), level=(3,4)), ['ppl_wgh',])

,ppl_wgh
gender_pair,
F,"(1.5359325982898095, 0.18515243423525063, 5)"
M,"(2.6112712945867083, 0.07959767494087383, 3)"
X,"(2.834075144088852, 0.03649952307498855, 5)"


### Comparison of gender pairs

In [55]:
ana.compare_gps(df_mf_raw.xs((0,0), level=(3,4)), ['dsim_wgh_nrm',])

,dsim_wgh_nrm
comp,
F:M,"(-2.1449098715433834, 0.06428295042964585, 8)"
F:X,"(0.04393121023237438, 0.9658240875606698, 10)"
M:X,"(1.0465369307837677, 0.3259002521870012, 8)"


In [56]:
ana.compare_gps(df_dir_raw.xs((0,0), level=(3,4)), ['dsim_wgh_nrm',])

,dsim_wgh_nrm
comp,
F:M,"(-6.370171467061146, 0.0002158982676641004, 8)"
F:X,"(0.4163847928990785, 0.6859243825050789, 10)"
M:X,"(2.4970816465949772, 0.03711041361649607, 8)"


In [57]:
ana.compare_gps(df_geo_raw.xs((0,0), level=(3,4)), ['dsim_wgh_nrm',])

,dsim_wgh_nrm
comp,
F:M,"(2.750898046055353, 0.02502417556429175, 8)"
F:X,"(2.87644996137917, 0.016485505869428697, 10)"
M:X,"(-0.8604491104409552, 0.4145830791282573, 8)"


In [58]:
ana.compare_gps(df_hdg_raw.xs((0,0), level=(3,4)), ['dsim_wgh_nrm',])

,dsim_wgh_nrm
comp,
F:M,"(-0.24227179189061385, 0.8146649021527637, 8)"
F:X,"(-0.2753287206829978, 0.7886689501201963, 10)"
M:X,"(-0.004045384761653405, 0.9968713169467593, 8)"


In [59]:
ana.compare_gps(df_ppl_raw.xs((0,0), level=(3,4)), ['ppl_wgh_nrm',])

,ppl_wgh_nrm
comp,
F:M,"(-0.22105022501072857, 0.830590859605987, 8)"
F:X,"(0.0009584130536858356, 0.9992541470166197, 10)"
M:X,"(0.3274734577772091, 0.7517148434115928, 8)"


### Influence of (individual) gender, role, and familiarity

In [60]:
ana.compare_genders(df_ppl_raw, df_bt, ['ppl_wgh_nrm'])

,ppl_wgh_nrm
comp,
f:m,"(-0.30879858808465566, 0.7596090715736816, 30)"


In [61]:
ana.compare_partner_genders(df_ppl_raw, df_bt, ['ppl_wgh_nrm'])

,ppl_wgh_nrm
comp,
f:m,"(-0.16700367169548488, 0.8684877650402874, 30)"


In [62]:
ana.compare_roles(df_ppl_raw, df_bt, ['ppl_wgh_nrm'])

,ppl_wgh_nrm
comp,
d:f,"(-0.3551885871891767, 0.7236538127822603, 62)"


In [63]:
ana.compare_roles(df_ppl_raw, df_bt, ['ppl_p', 'ppl_nrm_p'])

,ppl_p,ppl_nrm_p
comp,,
d:f,"(5.591866489183879, 5.382989729965968e-07, 62)","(3.2616946754706633, 0.0018023439554477195, 62)"


In [64]:
ana.compare_familiarities(df_mf_raw, df_bt, 'ses', ['dsim_wgh_nrm'])

,dsim_wgh_nrm
comp,
hi:lo,"(-0.18793075418958005, 0.8536276075129787, 14)"


In [65]:
ana.compare_familiarities(df_dir_raw, df_bt, 'ses', ['dsim_wgh_nrm'])

,dsim_wgh_nrm
comp,
hi:lo,"(-0.12256080200966889, 0.9041968815656958, 14)"


In [66]:
ana.compare_familiarities(df_geo_raw, df_bt, 'ses', ['dsim_wgh_nrm'])

,dsim_wgh_nrm
comp,
hi:lo,"(-0.15029528505713033, 0.8826755378561943, 14)"


In [67]:
ana.compare_familiarities(df_hdg_raw, df_bt, 'ses', ['dsim_wgh_nrm'])

,dsim_wgh_nrm
comp,
hi:lo,"(-0.3070666810531191, 0.763312588247354, 14)"


In [68]:
ana.compare_familiarities(
    df_ppl_raw, df_bt, 'ses', ['ppl_wgh_nrm'])

,ppl_wgh_nrm
comp,
hi:lo,"(3.333243355967619, 0.004925474067861711, 14)"


### Auxiliary tests and statistics

In [70]:
# total corpus duration
df_bt[df_bt['p_or_x'] != 'x']['duration'].sum() / 60

314.4158829085834

In [71]:
# average ipu duration
df_bt[df_bt['p_or_x'] != 'x']['duration'].mean()

1.5551028748260658

In [72]:
# ipu duration standard deviation
df_bt[df_bt['p_or_x'] != 'x']['duration'].std()

1.312417970190679

In [73]:
loc_cols = ['ses_id', 'end_time']
df_bt.loc[:,loc_cols].groupby('ses_id').max().mean()

end_time    1426.206361
dtype: float64

In [74]:
ana.check_chu_cnt_correlations(
    df_lsim.xs((0,0), level=(3,4)).iloc[4:], 
    df_bt, 'ses', True, stat_idx=3)

intensity_mean (0.11428887836108073, 0.6734208658463968, 14)
intensity_max (0.16213852887432906, 0.5485484607470928, 14)
pitch_mean (0.29628261629032043, 0.2651726894128137, 14)
pitch_max (0.18815497600212355, 0.48527632577555047, 14)
jitter (0.42271417680702583, 0.1028412959693719, 14)
shimmer (0.5794455155729361, 0.018654026293308815, 14)
nhr (0.42329514499645843, 0.10231928503548451, 14)
rate_syl (0.16133500703075843, 0.5505583950886497, 14)


In [75]:
ana.check_chu_cnt_correlations(
    df_syn.xs((0,0), level=(3,4)), 
    df_bt, 'ses', True, stat_idx=0, func=aux.r2z)

intensity_mean (-0.10123568349448873, 0.7091072114154436, 14)
intensity_max (0.04192689050468114, 0.8774756205639687, 14)
pitch_mean (0.16955262499166457, 0.5301564291999081, 14)
pitch_max (0.1497190947337907, 0.5799674638322942, 14)
jitter (0.2865944530404656, 0.2818529506908553, 14)
shimmer (0.49559403349094355, 0.05092075804009327, 14)
nhr (-0.14896153808153068, 0.581907890976349, 14)
rate_syl (0.2437055152877533, 0.363049992471346, 14)


In [76]:
ana.check_chu_cnt_correlations(
    df_lcon.xs((0,0), level=(3,4)), 
    df_bt, 'ses', True, stat_idx=0, func=aux.r2z)

intensity_mean (-0.23708967129092448, 0.3766243790532159, 14)
intensity_max (-0.2668191753838595, 0.31781095257082037, 14)
pitch_mean (-0.07120955646596958, 0.7932714738415021, 14)
pitch_max (-0.03488148801636415, 0.8979544210035557, 14)
jitter (-0.13882342724641908, 0.6081304781296658, 14)
shimmer (-0.18028502356247156, 0.5040371158591671, 14)
nhr (-0.19335754439027564, 0.47306162849124, 14)
rate_syl (0.2401930747314198, 0.3702225825313513, 14)


In [77]:
df_tmp = df_gsim_raw.xs(0, level=4).reset_index(level=3)
df_tmp = df_tmp[df_tmp['tsk_id'] != 0].set_index('tsk_id')
cols = [f + '_sim_p' for f in cfg.FEATURES]
ana.check_chu_cnt_correlations(df_tmp, df_bt, 'tsk', False, cols)

intensity_mean_sim_p (0.3472787348019726, 0.051482045341437625, 30)
intensity_max_sim_p (0.488821802224461, 0.004527091198382674, 30)
pitch_mean_sim_p (0.337846661238727, 0.05860499218807984, 30)
pitch_max_sim_p (0.3425105657147067, 0.05499165223007708, 30)
jitter_sim_p (0.48845268402884956, 0.004562190622207501, 30)
shimmer_sim_p (0.23498046061409628, 0.19546505482339854, 30)
nhr_sim_p (-0.06716280477709106, 0.7149418449939939, 30)
rate_syl_sim_p (0.6757847371732905, 2.1917520487283224e-05, 30)


In [78]:
df_tmp = df_gcon_raw.xs(0, level=4).reset_index(level=3)
df_tmp = df_tmp[df_tmp['tsk_id'] != 0].set_index('tsk_id')
cols = [f + '_con' for f in cfg.FEATURES]
ana.check_chu_cnt_correlations(df_tmp, df_bt, 'tsk', False, cols)

intensity_mean_con (0.019039835933915235, 0.9176220189554394, 30)
intensity_max_con (0.03139280198352095, 0.864569006319667, 30)
pitch_mean_con (0.2271453812526483, 0.21121705550685788, 30)
pitch_max_con (0.1801356566539743, 0.32385822976592116, 30)
jitter_con (-0.2670287146825051, 0.1395622024426189, 30)
shimmer_con (-0.10554456059420349, 0.5653550969053242, 30)
nhr_con (0.09131070338003572, 0.6191741613922497, 30)
rate_syl_con (-0.05566384384420389, 0.7622030821622066, 30)


In [79]:
ana.check_chu_cnt_correlations(
    df_mf_raw.xs((0,0), level=(3,4)), 
    df_bt, 'ses', False, ['dsim_p'])

dsim_p (0.5723818417268431, 0.020500190774166926, 14)


In [80]:
ana.check_chu_cnt_correlations(
    df_dir_raw.xs((0,0), level=(3,4)), 
    df_bt, 'ses', False, ['dsim_p'])

dsim_p (0.1996462455828926, 0.4585003175408411, 14)


In [81]:
ana.check_chu_cnt_correlations(
    df_geo_raw.xs((0,0), level=(3,4)), 
    df_bt, 'ses', False, ['dsim_p'])

dsim_p (-0.05086334943005576, 0.8516060715236315, 14)


In [82]:
ana.check_chu_cnt_correlations(
    df_hdg_raw.xs((0,0), level=(3,4)), 
    df_bt, 'ses', False, ['dsim_p'])

dsim_p (0.06923576843561219, 0.7988906633098519, 14)


In [83]:
ana.check_chu_cnt_correlations(
    df_ppl_raw.xs((0,0), level=(3,4)), 
    df_bt, 'ses', False, ['ppl_p'])

ppl_p (-0.03855489123709446, 0.8872686576556628, 14)


In [84]:
# token count per session, task, and speaker, with some meta-data
grp_cols = [
    'ses_id', 'tsk_id', 'task_index', 'speaker_a_or_b', 'familiarity']
loc_cols = grp_cols + ['speaker_role']
df = df_bt[df_bt['p_or_x'] != 'x']
df = df.loc[:,loc_cols].groupby(grp_cols).first().reset_index()
func1 = lambda x: lex.get_token_count(
    corpus_id, 
    'ses',
    x['ses_id'],
    x['speaker_a_or_b'])
df['ses_token_count'] = df.apply(func1, axis=1)
func2 = lambda x: lex.get_token_count(
    corpus_id, 
    'tsk',
    x['tsk_id'],
    x['speaker_a_or_b'])
df['tsk_token_count'] = df.apply(func2, axis=1)
fltr1 = df['speaker_role'] == 'd'
fltr2 = df['task_index'] == 1
fltr3 = df['familiarity'] == 'hi'

# test 1: describer token count vs. follower token count
print('d:f', aux.ttest_rel(df[fltr1]['tsk_token_count'], 
                           df[~fltr1]['tsk_token_count']))
# test 2: hi familiarity token count vs. lo familiarity token count
print('hi:lo', aux.ttest_ind(df[fltr2&fltr3]['tsk_token_count'], 
                             df[fltr2&~fltr3]['tsk_token_count']))

d:f (8.061753825694739, 4.1960321709523165e-09, 31)
hi:lo (-0.30819562589265476, 0.7600633170547341, 30)
